In [1]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred

In [2]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [3]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [4]:
sql="""SELECT s.ticker, max(d.stock_id), max(d.date_price)
FROM public.symbol s inner join daily_data d on s.id= d.stock_id
group by s.ticker"""
df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

In [5]:
df_ticker_last_day

,ticker,max,max
0,BOM500087,425,2020-03-27
1,BOM500010,423,2020-03-27
2,BOM500034,424,2020-03-27


In [6]:
sql="""select date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data where stock_id=71"""
df_stock=pd.read_sql(sql,con=conn_secmaster)

In [7]:
import datetime

In [8]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'level', 'anomaly']

In [9]:
symbol_id='BOM500034'
start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
indicator_name = 'candle_1'

In [10]:
sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data d join symbol s on d.stock_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
df_price=pd.read_sql(sql,con=conn_secmaster)

In [11]:
sql="select d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, indicator_name, start_date)
df_indicator=pd.read_sql(sql,con=conn_indicator)

In [12]:
df_indicator.set_index('date',inplace=True)

In [13]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    df_indicator.set_index('date',inplace=True)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
    if df_all_ind.empty:
        df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')

candle_1
candle_2
candle_3
level
anomaly


In [114]:
def level_plot(df):
    support_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]
    resistance_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_resistance']]
    end_dt=df.index[-1]
    res_plot_ls=[]
    sup_plot_ls=[]
    for res in resistance_ls:
        res_plot_ls.append(dict(x0=res[2],x1=end_dt,y0=res[0],y1=res[1],yref='y1',opacity=.2,fillcolor='Red',line=dict(color="black",width=1)))
    for sup in support_ls[:5]:
        sup_plot_ls.append(dict(x0=sup[2],x1=end_dt,y0=sup[0],y1=sup[1],yref='y1',opacity=.2,fillcolor='green',line=dict(color="black",width=1)))
    return (res_plot_ls+sup_plot_ls)

In [115]:
df=df_all_ind.set_index('date')
interested_feature='candle_1_level'

In [118]:
# Plotting
data = [ dict(
    type = 'candlestick',
    open = df.open,
    high = df.high,
    low = df.low,
    close = df.close,
    x = df.index,
    yaxis = 'y1',
    name = 'price'
)]

data.append( dict( x=df.index, y=df.volume,                         
                         marker=dict( color='blue' ),
                         type='bar', yaxis='y2', name='Volume'))

data.append( dict( x=df.index, y=df[interested_feature],                         
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y3', name=interested_feature))

layout=dict()    
layout['xaxis'] = dict( rangeslider = dict( visible = False ),autorange=True,fixedrange=False,visible=False,type='category')#type='category',
layout['yaxis'] = dict( domain = [0.2, 1],autorange = True,fixedrange=False)
layout['yaxis2'] = dict( domain = [0.0, 0.1],autorange = True,fixedrange=False)
layout['yaxis3'] = dict( domain = [0.1, 0.2],autorange = True,fixedrange=False)
# layout['shapes'] = [dict(x0=start_dt, x1=end_dt, y0=3057.9, y1=3060.85, yref='y1',opacity=.2,line=dict(color="Red",width=1))]#fillcolor='green',line_width=2
layout['shapes'] = level_plot(df)

df.to_csv("stock_price_indicator.csv",index=True)

fig = dict( data=data, layout=layout )

In [119]:
figure=go.FigureWidget(data=data, layout=layout)

In [120]:
def zoom(layout, x_range):
    print(x_range)
    in_view = df.iloc[int(figure.layout.xaxis.range[0]):int(figure.layout.xaxis.range[1])]
    figure.layout.yaxis2.range = [in_view.volume.min(), in_view.volume.max()]
    figure.layout.yaxis3.range = [in_view[interested_feature].min(), in_view[interested_feature].max()]

In [121]:
figure.layout.on_change(zoom, 'xaxis.range')

In [122]:
figure.update_layout(margin=dict(l=20, r=10),paper_bgcolor="LightSteelBlue",width=2200, height=1000)

FigureWidget({
    'data': [{'close': array([1722.1 , 1714.25, 1753.35, ..., 3080.55, 3091.4 , 3094.25]),
    …